# Visualize data on an interactive map

This notebook uses `lonboard` for interactive visualisation of data.

In [1]:
import geopandas as gpd
import numpy as np
import lonboard
from core.utils import used_keys
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify
from sidecar import Sidecar

Define data path

In [2]:
chars_dir = "/data/uscuni-ulce/processed_data/chars/"

Define region

In [3]:
country = 'fr_sp_nl_be'

In [4]:
region_hulls = gpd.read_parquet('/data/uscuni-ulce/' + "regions/" + f"{country}_regions_hull.parquet")

In [5]:
region_hulls

,convex_hull
labels,
153563,"POLYGON ((1561900 1019000, 1561500 1019200, 15..."
153635,"POLYGON ((1602000 1102800, 1602000 1103200, 16..."
153755,"POLYGON ((1694200 1032400, 1646700 1032500, 16..."
153834,"POLYGON ((1708000 1011300, 1698500 1011500, 16..."
153898,"POLYGON ((1707700 1037200, 1707900 1039400, 17..."
...,...
534115,"POLYGON ((4164800 2843200, 4146900 2851600, 41..."
534412,"POLYGON ((4203400 2086900, 4201600 2117000, 42..."
534524,"POLYGON ((4258700 2026300, 4245100 2032800, 42..."


In [6]:
m = region_hulls.explore()
region_hulls.loc[[157023]].explore(m=m, color='red')

In [6]:
region = 523438

## Buildings
Load building data and ensure the geometries are all valid Polygons.

In [13]:
buildings = (
    gpd.read_parquet(f"{chars_dir}buildings_chars_{region}.parquet")
    # .to_crs(4326)
    .reset_index()
)

In [14]:
buildings

,index,hauteur,date_d_apparition,beginning,end,currentUse,floor_area,longest_ridge_line_height,year_of_construction,x,...,mibFR,mibSCo,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbCeA,nID,stbSAl,nodeID
0,0,7.1,None,None,None,None,NaN,NaN,NaN,4.074289e+06,...,3.534709,3.534709,0.714286,13.600839,24.118625,25.571227,3.588619,7588.0,36.901057,10875.0
1,1,10.9,None,None,None,None,NaN,NaN,NaN,4.089195e+06,...,4.032426,4.032426,1.000000,9.473168,26.608234,44.808716,0.906213,25143.0,0.780502,26425.0
2,2,2.9,1900-01-01,None,None,None,NaN,NaN,NaN,4.094213e+06,...,3.429879,3.429879,0.800000,5.508635,9.249453,25.848839,0.111944,4754.0,13.514299,6300.0
3,3,7.8,None,None,None,None,NaN,NaN,NaN,4.094202e+06,...,3.429879,3.429879,0.625000,7.659844,15.103698,22.557499,1.013223,4753.0,15.576274,6300.0
4,4,2.8,None,None,None,None,NaN,NaN,NaN,4.065497e+06,...,1.588395,1.588395,0.800000,15.359300,12.923100,51.876631,23.053677,8642.0,19.486388,12414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509339,509339,4.1,None,None,None,None,NaN,NaN,NaN,4.060535e+06,...,2.292415,2.292415,0.833333,4.365382,58.931527,54.332907,4.881730,2789.0,4.050940,3977.0
509340,509340,5.7,None,None,None,None,NaN,NaN,NaN,4.060650e+06,...,3.464946,3.464946,1.000000,6.595432,57.555612,47.561499,21.806536,2789.0,5.093401,3977.0
509341,509341,NaN,None,None,None,None,NaN,NaN,NaN,4.060423e+06,...,1.008172,1.008172,0.800000,2.756030,27.481861,49.969019,1.997571,2789.0,14.009507,3977.0
509342,509342,6.8,None,None,None,None,NaN,NaN,NaN,4.076046e+06,...,7.644595,7.644595,0.500000,0.803418,7.907881,14.622119,13.491768,490.0,5.019403,688.0


In [75]:
# buildings = (
#     gpd.read_parquet(f"{chars_dir}buildings_chars_{region}.parquet")
#     # .to_crs(4326)
#     .reset_index()
# )

# buildings.geometry = buildings.make_valid()

# buildings = buildings[buildings.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [16]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(buildings, opacity=0.3)

/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 2.37 s, sys: 89 ms, total: 2.46 s
Wall time: 2.46 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [17]:
sc = Sidecar(title="buildings")

Create a Map object

In [18]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [19]:
with sc:
    display(m)

List avaialable columns

In [10]:
buildings.columns

Index(['index', 'gml_id', 'description', 'beginLifespanVersion',
       'conditionOfConstruction', 'localId', 'namespace', 'versionId',
       'referenceGeometry', 'horizontalGeometryEstimatedAccuracy', 'x', 'y',
       'id', 'iid', 'geometry', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbCCM',
       'ssbCCD', 'sdbAre', 'sdbPer', 'sdbCoA', 'ssbERI', 'ssbElo', 'stbOri',
       'mtbSWR', 'libNCo', 'ldbPWL', 'mibCou', 'mibLen', 'mibAre', 'mibElo',
       'mibERI', 'mibCCo', 'mibLAL', 'mibFR', 'mibSCo', 'ltcBuA', 'mtbAli',
       'mtbNDi', 'ltbIBD', 'stbCeA', 'nID', 'stbSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [11]:
# buildings.explore()

In [20]:
column = "mtbSWR"
used_keys[column]

'shared walls ratio of buildings'

In [21]:
classifier = classify(buildings[column], "quantiles", k=50)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps["viridis"])

/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/mapclassify/classifiers.py:1760: UserWarning: Not enough unique values in array to form 50 classes. Setting k to 31.
  self.bins = quantile(y, k=k)


In [13]:
buildings[column].describe().iloc[1:]

mean    0.698909
std     0.189620
min     0.038621
25%     0.569519
50%     0.718208
75%     0.852572
max     0.999996
Name: mibElo, dtype: float64

## Tessellation

Load tessellation data and ensure the geometries are all valid Polygons.

In [4]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)

tess.geometry = tess.make_valid()

tess = tess[tess.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [5]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(tess, opacity=0.2)

CPU times: user 4.89 s, sys: 341 ms, total: 5.23 s
Wall time: 5.24 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [14]:
sc = Sidecar(title="tess")

Create a Map object

In [7]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [15]:
with sc:
    display(m)

List avaialable columns

In [9]:
tess.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'barea', 'micBAD',
       'licBAD', 'stcSAl', 'nID', 'nodeID'],
      dtype='object')

In [12]:
column = "barea"
used_keys[column]

KeyError: 'barea'

Specify a column and pass its values into a choropleth representation within the map. 

In [13]:
classifier = classify(tess[column], "quantiles", k=40)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps["viridis"])

## Enclosures

Load data and ensure the geometries are all valid Polygons.

In [35]:
enc = gpd.read_parquet(f"{chars_dir}enclosures_chars_{region}.parquet").to_crs(4326)

enc.geometry = enc.make_valid()

enc = enc[enc.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [36]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(enc, opacity=0.3)

CPU times: user 168 ms, sys: 31.9 ms, total: 200 ms
Wall time: 200 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [37]:
sc = Sidecar(title="enclosures")

Create a Map object

In [38]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [39]:
with sc:
    display(m)

List avaialable columns

In [40]:
enc.columns

Index(['eID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWCe', 'likWBB'],
      dtype='object')

In [41]:
column = "ldkPer"
used_keys[column]

'perimeter of enclosure'

Specify a column and pass its values into a choropleth representation within the map. 

In [42]:
classifier = classify(enc[column], "quantiles", k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps["viridis"])

## Streets

Load data and ensure the geometries are all valid Polygons.

In [22]:
streets = gpd.read_parquet(f"{chars_dir}streets_chars_{region}.parquet")

streets.geometry = streets.make_valid()

Create a lonboard layer

In [23]:
%%time
layer = lonboard.PathLayer.from_geopandas(streets.to_crs(4326), width_min_pixels=1)

CPU times: user 154 ms, sys: 11.9 ms, total: 166 ms
Wall time: 165 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [24]:
sc = Sidecar(title="streets")

Create a Map object

In [25]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [26]:
with sc:
    display(m)

List avaialable columns

In [23]:
assert np.allclose(streets["sdsLen"], streets.geometry.length)

if "mm_len" in streets.columns:
    assert np.allclose(streets["mm_len"], streets.geometry.length)

In [24]:
streets.columns

Index(['geometry', '_status', 'mm_len', 'cdsbool', 'node_start', 'node_end',
       'sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre', 'sisBpM',
       'misBAD', 'sdsSPW', 'sdsSPO', 'sdsSWD', 'nID'],
      dtype='object')

In [25]:
streets[
    [
        "sdsLen",
        "sssLin",
        "ldsMSL",
        "sdsAre",
        "ldsRea",
        "ldsAre",
        "sisBpM",
        "sdsSPW",
        "sdsSPO",
        "sdsSWD",
    ]
].describe()

,sdsLen,sssLin,ldsMSL,sdsAre,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD
count,24155.000000,24155.000000,24155.000000,1.968500e+04,24155.000000,2.245400e+04,19046.000000,24155.000000,24155.000000,17632.000000
mean,386.031895,0.957977,399.593607,5.395904e+04,264.258456,9.260281e+05,0.063553,36.597900,0.821258,3.913480
std,629.662917,0.099379,349.484916,9.218028e+04,175.471539,9.230841e+05,0.052849,11.300294,0.183244,1.956093
min,1.779955,0.000000,22.997029,2.226503e-01,0.000000,3.801642e+03,0.000167,0.000000,0.050000,0.000000
25%,98.983143,0.970070,169.476379,8.441501e+03,147.000000,3.116659e+05,0.025527,26.756167,0.687500,2.555375
50%,193.610800,0.997328,250.932766,2.240300e+04,242.000000,6.098273e+05,0.052691,37.182577,0.869919,4.062591
75%,377.847408,0.999998,504.787106,5.820301e+04,354.000000,1.195124e+06,0.089664,50.000000,1.000000,5.386908
max,16951.254801,1.000000,6126.390497,1.758182e+06,1377.000000,8.037908e+06,1.846862,50.000000,1.000000,11.008117


Specify a column and pass its values into a choropleth representation within the map. 

In [30]:
column = "ldsAre"
used_keys[column]

'reached total ETC area by local street network'

In [31]:
streets[column] = streets[column].fillna(0)

In [33]:
classifier = classify(streets[column].astype(int), "quantiles", k=50)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps["viridis"])

## Nodes

Load data and ensure the geometries are all valid Polygons.

In [32]:
nodes = gpd.read_parquet(f"{chars_dir}nodes_chars_{region}.parquet").to_crs(4326)

Create a lonboard layer

In [33]:
%%time
layer = lonboard.ScatterplotLayer.from_geopandas(nodes, radius_min_pixels=2)

CPU times: user 33.8 ms, sys: 2.97 ms, total: 36.8 ms
Wall time: 36.5 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [34]:
sc = Sidecar(title="nodes")

Create a Map object

In [35]:
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)

Display map within the sidecar plugin

In [36]:
with sc:
    display(m)

List avaialable columns

In [37]:
nodes.columns

Index(['x', 'y', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'midBAD', 'sddAre', 'midRea', 'midAre'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [38]:
nodes[
    [
        "mtdDeg",
        "lcdMes",
        "linP3W",
        "linP4W",
        "linPDE",
        "lcnClo",
        "lddNDe",
        "linWID",
        "ldsCDL",
        "xcnSCl",
        "mtdMDi",
        "nodeID",
        "geometry",
        "sddAre",
        "midRea",
        "midAre",
    ]
].describe()

,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,nodeID,sddAre,midRea,midAre
count,19496.000000,19496.000000,19496.000000,19496.000000,19496.000000,1.949600e+04,19496.000000,19496.000000,19496.000000,19496.000000,19496.000000,19496.00000,1.711600e+04,19496.000000,1.765900e+04
mean,2.477944,0.085284,0.682505,0.104283,0.212255,1.887256e-06,0.003526,0.005831,882.309517,0.021187,386.081412,9747.50000,6.205794e+04,44.577247,1.552534e+05
std,1.014198,0.054032,0.118264,0.083171,0.110212,1.405456e-06,0.002425,0.003555,1289.245276,0.047371,518.177763,5628.15476,9.149249e+04,41.710329,1.992186e+05
min,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000286,0.000000,0.000000,0.000000,3.099139,0.00000,6.428662e+00,0.000000,2.219230e+02
25%,1.000000,0.047059,0.622951,0.045455,0.142857,7.424187e-07,0.001678,0.002700,229.506759,0.000000,133.671630,4873.75000,1.330871e+04,14.000000,3.685127e+04
50%,3.000000,0.081081,0.695652,0.090909,0.200000,1.530668e-06,0.003297,0.005482,500.392489,0.000000,210.929432,9747.50000,3.054813e+04,35.000000,8.467917e+04
75%,3.000000,0.117647,0.760000,0.146341,0.266667,2.713025e-06,0.004740,0.008229,993.203161,0.000000,395.362400,14621.25000,7.022078e+04,63.000000,1.880395e+05
max,5.000000,0.352941,1.000000,0.550000,1.000000,9.021213e-06,0.062490,0.048315,26019.573144,0.500000,16951.254801,19495.00000,1.280031e+06,457.000000,2.196893e+06


In [39]:
column = "mtdMDi"
used_keys[column]

'mean distance to neighbouring nodes of street network'

In [40]:
classifier = classify(nodes[column], "quantiles", k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps["viridis"])

## Visualize merged data

In [4]:
import pandas as pd

primary = pd.read_parquet(chars_dir + f"primary_chars_{region}.parquet")

In [5]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)
tess.geometry = tess.make_valid()
tess = tess[tess.geom_type.str.contains("Polygon")]
tess = tess[["enclosure_index", "geometry"]]

In [6]:
tess = tess.join(primary)

In [7]:
tess

,enclosure_index,geometry,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,...,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,licBAD,misBAD,midBAD
0,4292,"POLYGON ((6.02685 47.87079, 6.02688 47.87084, ...",5.211977,9.615898,0.0,0.516584,4,1.786527,0.997946,0.529904,...,0.557318,1.014509,0.551741,3.784041,0.605716,0.605716,47.986063,123.897282,476.535688,476.535688
1,4292,"POLYGON ((6.02785 47.87357, 6.02789 47.87356, ...",104.209279,40.852077,0.0,0.629445,4,1.071267,0.999771,0.957660,...,0.651697,1.005446,0.587735,18.624055,3.109076,3.109076,35.618158,39.326087,476.535688,476.535688
2,4292,"POLYGON ((6.02715 47.87263, 6.0272 47.87265, 6...",50.782876,30.273110,0.0,0.500592,4,0.922463,0.999261,0.498341,...,0.651697,1.005446,0.587735,18.624055,3.109076,3.109076,37.103967,123.377611,476.535688,476.535688
3,4292,"POLYGON ((6.02748 47.8715, 6.0272 47.87141, 6....",13.823494,15.052444,0.0,0.594648,4,2.344267,0.997836,0.754407,...,0.765952,1.008661,0.614764,5.640245,0.979567,0.979567,44.698739,118.787262,476.535688,476.535688
4,4292,"POLYGON ((6.02671 47.87077, 6.02668 47.87077, ...",129.923899,45.885153,0.0,0.617143,4,0.340736,0.999808,0.800433,...,0.803507,1.003479,0.623756,16.572175,2.892592,2.892592,98.249864,122.672534,476.535688,476.535688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25078,3615,"POLYGON ((2.64609 42.46739, 2.64627 42.46746, ...",71.751467,33.902475,0.0,0.632969,4,0.324528,1.000032,0.931890,...,0.198499,0.922816,0.205281,53.392635,3.677282,3.677282,60.409994,82.414342,98.908209,98.908209
25079,3615,"POLYGON ((2.64609 42.46739, 2.64602 42.46744, ...",89.244424,37.788165,0.0,0.632787,4,1.068803,1.002729,0.862392,...,0.198499,0.922816,0.205281,53.392635,3.677282,3.677282,55.973632,83.733350,98.908209,98.908209
25080,3615,"POLYGON ((2.64591 42.4675, 2.64589 42.46751, 2...",108.038520,42.487980,0.0,0.580735,4,0.436655,0.999709,0.660232,...,0.198499,0.922816,0.205281,53.392635,3.677282,3.677282,67.063675,81.179534,98.908209,98.908209
25081,3651,"POLYGON ((2.64168 42.4683, 2.64167 42.46829, 2...",145.836191,59.538635,0.0,0.367755,10,0.815958,0.874408,0.456723,...,0.461974,0.882824,0.376112,22.670134,2.528546,2.528546,24.604972,24.604972,98.908209,98.908209


Create a lonboard layer

In [8]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(tess, opacity=0.2)

CPU times: user 167 ms, sys: 16.3 ms, total: 183 ms
Wall time: 184 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [9]:
sc = Sidecar(title="tess")

Create a Map object

In [10]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [11]:
with sc:
    display(m)

List avaialable columns

In [12]:
tess.columns

Index(['enclosure_index', 'geometry', 'sdbAre', 'sdbPer', 'sdbCoA', 'ssbCCo',
       'ssbCor', 'ssbSqu', 'ssbERI', 'ssbElo', 'ssbCCM', 'ssbCCD', 'stbOri',
       'mtbSWR', 'libNCo', 'ldbPWL', 'ltcBuA', 'mtbAli', 'mtbNDi', 'ltbIBD',
       'stbCeA', 'stbSAl', 'sdsLen', 'sssLin', 'ldsMSL', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD', 'mtdDeg', 'lcdMes', 'linP3W',
       'linP4W', 'linPDE', 'lcnClo', 'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl',
       'mtdMDi', 'sddAre', 'midRea', 'midAre', 'stcOri', 'sdcLAL', 'sdcAre',
       'sscCCo', 'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl',
       'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB',
       'likWBB', 'sdsAre', 'likWCe', 'mibCou', 'mibAre', 'mibLen', 'mibElo',
       'mibERI', 'mibCCo', 'mibLAL', 'mibFR', 'mibSCo', 'micBAD', 'licBAD',
       'misBAD', 'midBAD'],
      dtype='object')

In [36]:
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"
from libpysal.graph import read_parquet

graph = read_parquet(graph_dir + f"tessellation_graph_{region}.parquet")
graph3 = graph.higher_order(k=3, lower_order=True, diagonal=True)

In [45]:
tess["mibElo3"] = graph3.describe(tess["mibElo"])["median"]

In [50]:
tess["mibERI3"] = graph3.describe(tess["mibERI"])["median"]

In [13]:
column = "lskERI"
used_keys[column]

'equivalent rectangular index of enclosure'

In [14]:
tess[column].describe()

count    25083.000000
mean         0.706487
std          0.259224
min          0.281331
25%          0.502071
50%          0.667595
75%          0.949421
max          1.128042
Name: lskERI, dtype: float64

Specify a column and pass its values into a choropleth representation within the map. 

In [15]:
classifier = classify(tess[column], "quantiles", k=100)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps["viridis"])

/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/mapclassify/classifiers.py:1760: UserWarning: Not enough unique values in array to form 100 classes. Setting k to 86.
  self.bins = quantile(y, k=k)


In [24]:
## ldsAre and sdsAre, ldsRea seperate atleast somewhat apartments and industrial,
# maybe add sdsRea or
# the same stuff except for connected buildings
# or reached unique streets
# mtdMDi has potential, but we have to change it to something like distance to nearest 8 nodes or something
# square clustering does nothing, looks weird on the graph, probably something with grids...
# ldsCDL okish
# linWID oksih
# lddNDe okish
# 'local closeness of street network' suprisingly ok, but not enough variability to do anything
# meshedness bad
# degree useless

# ldbPWL - miht make thigs worse
# street profile is not great for this
# mtbNDi meh
# ltbIBD meh


# midrea and mid area are meh, stds might be more usefull
# cell area, lal not usefull
# mtcWNe not useful
# mdcAre - meh
# ltcWRB - meh
